# **15.3 Encapsulation_and_Properties**

Encapsulation means hiding internal details and controlling access to data through methods. Instead of allowing direct changes to `pokemon.hp = -999`, you use properties and validation to maintain data integrity. In this lesson you'll learn private attributes (name mangling with `_`), the `@property` decorator, getters and setters, and how to build Pokemon classes that protect their own data.

---

## **The Problem: Direct Attribute Access**

Without encapsulation, anyone can set attributes to invalid values, breaking your object's invariants.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self.name = name
        self.level = level
        self.hp = hp
        self.max_hp = hp

pikachu = Pokemon("Pikachu", 25, 35)

# Nothing stops this!
pikachu.hp = -999      # Invalid HP
pikachu.level = 0      # Invalid level
pikachu.name = ""      # Empty name

print(f"{pikachu.name} Lv.{pikachu.level} HP:{pikachu.hp}")
print("Data is corrupted! We need validation.")

---

## **Convention: Private Attributes with _**

Python uses naming conventions to indicate internal attributes. A single underscore `_attribute` means "private — don't access directly". Python won't stop you, but it's a signal to other programmers.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self._name = name        # "Private" by convention
        self._level = level
        self._hp = hp
        self._max_hp = hp
    
    def get_hp(self):
        """Getter method for HP."""
        return self._hp
    
    def set_hp(self, value):
        """Setter method with validation."""
        if value < 0:
            print(f"Warning: HP cannot be negative, setting to 0")
            self._hp = 0
        elif value > self._max_hp:
            print(f"Warning: HP cannot exceed max, setting to {self._max_hp}")
            self._hp = self._max_hp
        else:
            self._hp = value

pikachu = Pokemon("Pikachu", 25, 35)

# Access through methods
print(f"HP: {pikachu.get_hp()}")
pikachu.set_hp(-999)  # Validation kicks in
print(f"HP: {pikachu.get_hp()}")
pikachu.set_hp(100)   # Also validated
print(f"HP: {pikachu.get_hp()}")

---

## **The @property Decorator**

Using `get_hp()` and `set_hp()` is verbose. The `@property` decorator lets you access methods like attributes while keeping validation.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self._name = name
        self._level = level
        self._hp = hp
        self._max_hp = hp
    
    @property
    def hp(self):
        """Getter for HP — accessed like pokemon.hp"""
        return self._hp
    
    @hp.setter
    def hp(self, value):
        """Setter for HP — set like pokemon.hp = value"""
        if value < 0:
            self._hp = 0
        elif value > self._max_hp:
            self._hp = self._max_hp
        else:
            self._hp = value

pikachu = Pokemon("Pikachu", 25, 35)

# Now we can use it like a regular attribute, but with validation!
print(f"HP: {pikachu.hp}")   # Calls the getter
pikachu.hp = -999              # Calls the setter (validated)
print(f"HP: {pikachu.hp}")
pikachu.hp = 100               # Also validated
print(f"HP: {pikachu.hp}")

---

## **Read-Only Properties**

If you define only `@property` without a setter, the attribute becomes read-only.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self._name = name
        self._level = level
        self._hp = hp
        self._max_hp = hp
    
    @property
    def name(self):
        """Read-only name."""
        return self._name
    
    @property
    def hp_percentage(self):
        """Computed read-only property."""
        return (self._hp / self._max_hp) * 100

pikachu = Pokemon("Pikachu", 25, 35)

# Can read
print(f"Name: {pikachu.name}")
print(f"HP: {pikachu.hp_percentage:.1f}%")

# Cannot write
try:
    pikachu.name = "Raichu"  # AttributeError
except AttributeError as e:
    print(f"Error: {e}")

---

## **Computed Properties**

Properties can calculate values on the fly instead of storing them. This keeps data always in sync.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self._name = name
        self._level = level
        self._hp = hp
        self._max_hp = hp
    
    @property
    def hp(self):
        return self._hp
    
    @hp.setter
    def hp(self, value):
        self._hp = max(0, min(self._max_hp, value))
    
    @property
    def is_fainted(self):
        """Computed — always accurate."""
        return self._hp == 0
    
    @property
    def hp_percentage(self):
        """Computed — always accurate."""
        return (self._hp / self._max_hp) * 100
    
    @property
    def status(self):
        """Computed status based on HP."""
        if self.is_fainted:
            return "Fainted"
        elif self.hp_percentage < 25:
            return "Critical"
        elif self.hp_percentage < 50:
            return "Low"
        else:
            return "Healthy"

pikachu = Pokemon("Pikachu", 25, 35)

for hp in [35, 20, 10, 5, 0]:
    pikachu.hp = hp
    print(f"HP: {pikachu.hp:2} ({pikachu.hp_percentage:5.1f}%) — Status: {pikachu.status}")

---

## **Validation in Setters**

Setters are the perfect place to validate data before accepting it.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self.name = name    # Uses setter
        self.level = level  # Uses setter
        self._hp = hp
        self._max_hp = hp
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError(f"Name must be str, got {type(value).__name__}")
        if not value.strip():
            raise ValueError("Name cannot be empty")
        self._name = value.strip()
    
    @property
    def level(self):
        return self._level
    
    @level.setter
    def level(self, value):
        if not isinstance(value, int):
            raise TypeError(f"Level must be int, got {type(value).__name__}")
        if not (1 <= value <= 100):
            raise ValueError(f"Level must be 1-100, got {value}")
        self._level = value

# Valid
pikachu = Pokemon("Pikachu", 25, 35)
print(f"{pikachu.name} Lv.{pikachu.level}")

# Invalid — caught immediately
try:
    pikachu.name = ""
except ValueError as e:
    print(f"Validation error: {e}")

try:
    pikachu.level = 150
except ValueError as e:
    print(f"Validation error: {e}")

---

## **Double Underscore: Name Mangling**

A double underscore `__attribute` triggers name mangling — Python transforms it to `_ClassName__attribute` to make it harder to access accidentally. This is for truly internal implementation details.

In [ ]:
class Pokemon:
    def __init__(self, name, secret_id):
        self.name = name
        self.__secret_id = secret_id  # Name mangled
    
    def reveal_secret(self):
        return f"Secret ID: {self.__secret_id}"

pikachu = Pokemon("Pikachu", "PKM-12345")

print(pikachu.name)  # Normal access

# Cannot access directly
try:
    print(pikachu.__secret_id)
except AttributeError as e:
    print(f"Error: {e}")

# Access through method
print(pikachu.reveal_secret())

# Still accessible if you really want (name mangled)
print(f"\nMangled name: {pikachu._Pokemon__secret_id}")
print("(But don't do this — it defeats the purpose!)")

---

## **Practical: Complete Encapsulated Pokemon**

In [ ]:
class Pokemon:
    """
    Fully encapsulated Pokemon with validation.
    """
    
    def __init__(self, name, ptype, level, hp, attack, defense):
        # Use setters for validation
        self.name = name
        self.type = ptype
        self.level = level
        self._hp = hp
        self._max_hp = hp
        self.attack = attack
        self.defense = defense
    
    # Name property with validation
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str) or not value.strip():
            raise ValueError("Name must be a non-empty string")
        self._name = value.strip()
    
    # Type property with validation
    @property
    def type(self):
        return self._type
    
    @type.setter
    def type(self, value):
        valid_types = {"Fire", "Water", "Grass", "Electric", "Normal"}
        if value not in valid_types:
            raise ValueError(f"Type must be one of {valid_types}")
        self._type = value
    
    # Level property with validation
    @property
    def level(self):
        return self._level
    
    @level.setter
    def level(self, value):
        if not isinstance(value, int) or not (1 <= value <= 100):
            raise ValueError("Level must be an integer 1-100")
        self._level = value
    
    # HP property with auto-clamping
    @property
    def hp(self):
        return self._hp
    
    @hp.setter
    def hp(self, value):
        self._hp = max(0, min(self._max_hp, value))
    
    # Attack property with validation
    @property
    def attack(self):
        return self._attack
    
    @attack.setter
    def attack(self, value):
        if not isinstance(value, int) or value < 1:
            raise ValueError("Attack must be a positive integer")
        self._attack = value
    
    # Defense property with validation
    @property
    def defense(self):
        return self._defense
    
    @defense.setter
    def defense(self, value):
        if not isinstance(value, int) or value < 1:
            raise ValueError("Defense must be a positive integer")
        self._defense = value
    
    # Computed read-only properties
    @property
    def is_fainted(self):
        return self._hp == 0
    
    @property
    def hp_percentage(self):
        return (self._hp / self._max_hp) * 100
    
    def __str__(self):
        return f"{self.name} ({self.type}) Lv.{self.level} HP:{self.hp}/{self._max_hp}"
    
    def take_damage(self, damage):
        """Reduce HP by damage amount."""
        self.hp -= damage  # Uses setter — automatically clamped
        if self.is_fainted:
            print(f"{self.name} fainted!")
    
    def heal(self, amount):
        """Restore HP."""
        old_hp = self.hp
        self.hp += amount  # Uses setter — automatically clamped
        healed = self.hp - old_hp
        print(f"{self.name} restored {healed} HP")

# Test validation
print("Creating valid Pokemon:")
pikachu = Pokemon("Pikachu", "Electric", 25, 35, 55, 40)
print(pikachu)

print("\nTesting HP clamping:")
pikachu.hp = -999
print(f"HP after -999: {pikachu.hp}")
pikachu.hp = 100
print(f"HP after 100: {pikachu.hp}")

print("\nTesting validation:")
try:
    pikachu.level = 150
except ValueError as e:
    print(f"Caught: {e}")

try:
    pikachu.type = "Dragon"
except ValueError as e:
    print(f"Caught: {e}")

print("\nComputed properties:")
print(f"Is fainted: {pikachu.is_fainted}")
print(f"HP percentage: {pikachu.hp_percentage:.1f}%")

---

## **Practice Exercises**

### **Task 1: Basic Property**

Create a property for `name` with getter and setter.

**Expected Output:**
```
Pikachu
```

In [ ]:
# Your code here:


### **Task 2: Read-Only Property**

Create a read-only `name` property (no setter).

**Expected Output:**
```
AttributeError: can't set attribute
```

In [ ]:
# Your code here:


### **Task 3: Computed Property**

Create a computed `hp_percentage` property.

**Expected Output:**
```
100.0%
```

In [ ]:
# Your code here:


### **Task 4: HP Validation**

Add HP property that clamps values to 0–max_hp.

**Expected Output:**
```
HP: 0 (clamped from -999)
HP: 35 (clamped from 100)
```

In [ ]:
# Your code here:


### **Task 5: Level Validation**

Add level property that raises ValueError if not 1-100.

**Expected Output:**
```
ValueError: Level must be 1-100
```

In [ ]:
# Your code here:


### **Task 6: Name Validation**

Validate that name is non-empty string.

**Expected Output:**
```
ValueError: Name cannot be empty
```

In [ ]:
# Your code here:


### **Task 7: is_fainted Property**

Add computed `is_fainted` property.

**Expected Output:**
```
True
```

In [ ]:
# Your code here:


### **Task 8: Private Attribute**

Use `_hp` as private attribute with property.

**Expected Output:**
```
HP: 35
```

In [ ]:
# Your code here:


### **Task 9: Type Validation**

Validate type is one of valid Pokemon types.

**Expected Output:**
```
ValueError: Invalid type 'Dragon'
```

In [ ]:
# Your code here:


### **Task 10: Complete Encapsulation**

Build a fully encapsulated Pokemon with validated properties.

**Expected Output:**
```
Pikachu (Electric) Lv.25 HP:35/35
All validation working!
```

In [ ]:
# Your code here:


---

## **Summary**

- Encapsulation hides internal details and controls access
- `_attribute` — convention for private (not enforced)
- `__attribute` — name mangling (harder to access)
- `@property` — creates getter method
- `@property_name.setter` — creates setter method
- Properties accessed like attributes: `obj.hp`
- Setters validate before accepting values
- Read-only properties have no setter
- Computed properties calculate on access
- Validation in setters maintains data integrity

---

## **Quick Reference**

```python
class Pokemon:
    def __init__(self, name, hp):
        self._name = name  # Private by convention
        self._hp = hp
        self._max_hp = hp
    
    # Read-write property
    @property
    def hp(self):
        """Getter."""
        return self._hp
    
    @hp.setter
    def hp(self, value):
        """Setter with validation."""
        self._hp = max(0, min(self._max_hp, value))
    
    # Read-only property
    @property
    def name(self):
        return self._name  # No setter → read-only
    
    # Computed property
    @property
    def hp_percentage(self):
        return (self._hp / self._max_hp) * 100

# Usage
p = Pokemon("Pikachu", 35)
print(p.hp)          # Calls getter
p.hp = 20            # Calls setter
print(p.hp_percentage)  # Computed
# p.name = "X"      # Error — read-only
```